In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
# loading env variables
from dotenv import  load_dotenv

load_dotenv()

KEY = os.getenv("OPENAI_API_KEY")


In [4]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.6)

d:\Machine learning\Generative AI\MCQ generator\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
# Importing libraries
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [6]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [7]:
TEMPLATE_1 = """
You are an expert in MCQ generation. Please generate {number} multiple choice questions about {subject} subject from the delimited text by triple double quotes and question should in {tone} tone. \

Text to generate questions:\
{text}

Please make sure that you don't forget about the question tone, it is really important to assess the student knowledge and exclude the delimiter.  If there is nothing related to {subject} subject then please show that \
\"There is no text related to {subject}\".\

Outputs should be in JSON  object following:
{response_json}

"""




In [8]:
# Defining prompt template

quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE_1
)

In [9]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [10]:
TEMPLATE_2 = """
You are experienced professor. You need to evaluate the complexity of multiple choice question delimited by triple double quotes and give a complete analysis of the quiz about {subject}.\
Only use maximum 50 words for complexity analysis. If the quiz is not suitable with the cognitive and analytical abilities of the {subject} students, update the quiz questions \
which needs to be changed and change the tone of the multiple choice questions such that perfectly aligns and fits the {subject} student abilities. Outputs should in JSON object \
that includes the following:\
keys: question, multiple choices, and correct answer

Quiz multiple choice question:
\"\"\"{quiz}\"\"\"

Check from an {subject} expert that the question and answer is valid and correct.
"""

In [11]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["quiz", "subject"],
    template = TEMPLATE_2
)

In [12]:
# Create chain to evaluate the quesitons in quiz
quiz_evaluation_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [13]:
# setting up the SequentialChain
complete_quiz_chain = SequentialChain(
    chains=[quiz_chain, quiz_evaluation_chain], 
    input_variables=["text", "number", "subject", "tone", "response_json"], 
    output_variables=["quiz", "review"],
    verbose=True
 )

In [14]:
# loading the text data
file_path = '../data.txt'
with open(file_path, 'r') as file:
    TEXT = file.read()

NUMBER = 10
SUBJECT = 'Physics'
TONE = 'Normal'

In [15]:
# Setup token usage tracking in langchain

with get_openai_callback() as cb:
    response = complete_quiz_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

d:\Machine learning\Generative AI\MCQ generator\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert in MCQ generation. Please generate 10 multiple choice questions about Physics subject from the delimited text by triple double quotes and question should in Normal tone. 
Text to generate questions:In sixth-century Europe John Philoponus, a Byzantine scholar, questioned Aristotle's teaching of physics and noted its flaws. He introduced the theory of impetus. Aristotle's physics was not scrutinized until Philoponus appeared; unlike Aristotle, who based his physics on verbal argument, Philoponus relied on observation. On Aristotle's physics Philoponus wrote:

But this is completely erroneous, and our view may be corroborated by actual observation more effectively than by any sort of verbal argument. For if you let fall from the same height two weights of which one is many times as heavy as the other, you will see that the ratio of the times required for the motion does

In [16]:
quiz=response.get("quiz")
quiz=json.loads(quiz)


In [17]:
import pandas as pd

quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

quiz=pd.DataFrame(quiz_table_data)


In [18]:
quiz.to_csv(f"{SUBJECT}.csv",index=False)
